# GC2 Jupyter Notebook eksempel

Jupyter Notebook er et webbaseret interaktivt analyse-miljø til oprettelse af notebook dokumenter. Et Jupyter Notebook-dokument er en skal, der indeholder en ordnet liste over input/outputceller, som kan indeholde kode, tekst (ved hjælp af Markdown), matematik, plots og media.   

Denne Notebook er et eksempel på brugen af databasen dk.gc2.io, der indeholde mange offentlige datasæt - herunder matrikelkortet, danske adresser, grukos, GeoDK og mange flere.  

Der bruges gc2-python-client, som gør det nemt at logge på, køre forespørgelser i databasen og få output formatteret. Til visualisering anvendes bl.a. Pandas til tabeller og Folium til kort.

## Install af gc2-python-client
Først skal gc2-python-client installeres. Det gøre med en Background processes. Dette er ikke kode, men en måde hvorpå, man kan installere Python pakker på.

In [ ]:
!pip install --index-url https://test.pypi.org/simple/ gc2-python-client

Derefter kan der kodes. Først importeres de nødvendige pakker.

In [4]:
import gc2
import folium
import pandas

Der initialiseres et Gc2 object, som gør det muligt at autentificere en bruger.

In [5]:
local = gc2.Gc2("https://dk.gc2.io")
local.set_authentication(user="dk_test", pw="Pwd1234!", db="dk")

Så initialiseres et Sql objekt, som gør det muligt at køre SQL statements

In [10]:
sql = gc2.Sql(local)
statement = "select ejerlavsnavn, matrikelnummer, 1 as hej, the_geom from matrikel.jordstykke limit 6"
sql.run(statement)


Når statement er kørt kan resultatet bruges. Resultat findes som en to-dimensionel "list" (en tabel) og kan bruges direkte i Pandas. Der er også en "list" med feltoverskrifter. 

In [11]:
pandas.DataFrame(data=sql.rows, columns=sql.columns)

,ejerlavsnavn,matrikelnummer,hej
0,"Kgs. Lyngby By, Kgs. Lyngby",9u,1
1,"Kgs. Lyngby By, Kgs. Lyngby",9m,1
2,"Kgs. Lyngby By, Kgs. Lyngby",9aq,1
3,"Kgs. Lyngby By, Kgs. Lyngby",2ca,1
4,"Kgs. Lyngby By, Kgs. Lyngby",7000z,1
5,"Kgs. Lyngby By, Christians",4c,1


Resultatet findes også som GeoJSON, der kan bruges i et Folium kort.

In [13]:
m = folium.Map(
    location=[56.1759, 10.6016],
    tiles="cartodbpositron",
    zoom_start=7,
)
folium.GeoJson(sql.geojson, name="Jordstykker").add_to(m)
folium.LayerControl().add_to(m)
m